<a href="https://colab.research.google.com/github/nrnrk/public-resources/blob/main/google-colaboratory/wc-cuda/wc_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## `wc` コマンドの CUDA 実装

学習のために `wc` コマンドを CUDA で実装し直します。大きいファイルでは本家より早くなることをごく簡単な計測で確認できました。

In [1]:
# リポジトリのクローン
!git clone https://github.com/nrnrk/public-resources.git
# 作業ディレクトリをリポジトリに移動
%cd public-resources/google-colaboratory/wc-cuda

Cloning into 'public-resources'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 0), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 6.42 KiB | 6.42 MiB/s, done.
/content/public-resources/google-colaboratory/wc-cuda


### 準備

* Colaboratory で `編集` > `ノートブックの設定` で CUDA 対応のアクセラレータ(T4など)を選択
  * これにより関連のミドルウェアはインストールされます
* `PyCUDA` のインストール


In [2]:
# 2分くらいかかります
!pip install pycuda

### .cubin の生成

`wc.cu` に CUDA カーネルを定義しています。実行速度を上げるために `.cubin` を事前に生成しておきます。

In [3]:
# T4 は sm_75 のアーキテクチャ
!nvcc -cubin -arch=sm_75 wc.cu -o wc.cubin

### wc.py を実行

`wc.cubin` をロードして、 `wc` コマンド相当のものを実行するコード `wc.py` を実行します。

In [4]:
!python wc.py wc.py

 79 208 2410 wc.py


### 時間の計測

`time` コマンドで簡易的に速度を計測する。テスト用の短いテキストと、長いテキストも事前に生成して、 `wc` コマンドと比較します。

In [5]:
# 短めのデータを生成
with open('short_text.txt', 'w') as f:
  for _ in range(100):
    f.write('short text!\n' * 5)

In [6]:
# とても長いデータを生成
with open('long_text.txt', 'w') as f:
  for _ in range(100000):
    f.write('super long text!\n' * 100)

In [7]:
!time wc short_text.txt

 500 1000 6000 short_text.txt

real	0m0.002s
user	0m0.000s
sys	0m0.002s


In [8]:
!time python wc.py short_text.txt

 500 1000 6000 short_text.txt

real	0m0.507s
user	0m0.205s
sys	0m0.242s


In [9]:
!time wc long_text.txt

 10000000  30000000 170000000 long_text.txt

real	0m1.877s
user	0m1.555s
sys	0m0.058s


In [10]:
!time python wc.py long_text.txt

 10000000 30000105 170000000 long_text.txt

real	0m0.530s
user	0m0.292s
sys	0m0.318s
